In [17]:
import cv2
import numpy as np
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pickle
from skimage.feature import local_binary_pattern

In [19]:
# --- 1. Feature Extraction Function (using Local Binary Patterns) ---
def extract_lbp_features(gray_image, radius=3, n_points=8 * 3):
    """Extracts Local Binary Pattern (LBP) features from a grayscale image."""
    lbp = local_binary_pattern(gray_image, n_points, radius, method='uniform')
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp.ravel(), density=True, bins=n_bins, range=(0, n_bins))
    return hist


def extract_features(image_path):
    """Loads an image, extracts LBP features, and returns the feature vector."""
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Could not load image at {image_path}")
            return None
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        features = extract_lbp_features(gray)
        return features
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None
    
    image_path = "D:/code_copy/500_dataset/500_s1.jpg"

In [22]:
# --- 2. Data Collection ---
features = []
labels = []
data_dir = "D:/code_copy/500_dataset"  # **<-- REPLACE WITH THE ACTUAL PATH TO YOUR IMAGE DIRECTORY**

if not os.path.exists(data_dir):
    print(f"Error: Directory not found at {data_dir}")
else:
    for label_name in ["real", "fake"]:
        label_dir = os.path.join(data_dir, label_name)
        if not os.path.exists(label_dir):
            print(f"Warning: Directory not found: {label_dir}")
            continue
        for img_file in os.listdir(label_dir):
            if img_file.endswith((".jpg", ".jpeg", ".png")):
                image_path = os.path.join(label_dir, img_file)
                feature_vector = extract_features(image_path)
                if feature_vector is not None:
                    features.append(feature_vector)
                    labels.append(1 if label_name == "real" else 0)

In [23]:
# --- 3. Convert to NumPy Arrays ---
if not features:
    print("Error: No features were extracted. Please check your image directory and feature extraction function.")
    exit()

X = np.array(features)
y = np.array(labels)

In [24]:
# --- 4. Data Splitting ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # Added stratify for better representation

In [25]:
# --- 5. Model Training ---
clf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced') # Increased n_estimators and added class_weight
clf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=200,
                       random_state=42)

In [26]:
# --- 6. Model Evaluation ---
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy on Test Set: {accuracy * 100:.2f}%")
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred))

Model Accuracy on Test Set: 75.00%

Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       1.00      0.50      0.67         2

    accuracy                           0.75         4
   macro avg       0.83      0.75      0.73         4
weighted avg       0.83      0.75      0.73         4



In [27]:
# --- 7. Save the Model ---
model_filename = 'currency_model_lbp.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(clf, model_file)
print(f"\nModel saved as {model_filename}")


Model saved as currency_model_lbp.pkl
